In [32]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from transformers import TFBertForSequenceClassification, BertTokenizer

### Training data, label ID's and model read

In [33]:
training_data = [
    {'text' : 'How is my financial health?', 'label': 'analyze_finances'},      # balance and financial tips
    {'text' : 'What is my balance?', 'label': 'check_balance'},
    {'text' : 'Give me tips to helpe me save up money', 'label': 'financial_tips'},

    {'text' : 'How can I open a new account?', 'label': 'open_account'},        # account management
    {'text' : 'Can I close my savings?', 'label' : 'close_account'},
    { 'text' : 'How do I update my contact information?', 'label' : 'update_account_info' },
    { 'text' : 'I want to change my account type.', 'label' : 'change_account_type' }
]


label_to_id = {
    'analyze_finances' : 0,          # balance and financial tips
    'check_balance' : 1,
    'financial_tips' : 2,

    'open_account' : 3,             # account management
    'close_account' : 4,
    'update_account_info' : 5,
    'change_account_type' : 6
}


# load a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
texts = [ item['text'] for item in training_data ]
labels = [  label_to_id[ item['label'] ] for item in training_data  ]

inputs = tokenizer( texts, padding=True, truncation=True, return_tensors='tf' )
labels = tf.constant( labels )

### Verifying that everything is working as it should

In [34]:
print("Input IDs:", inputs['input_ids'])
print("Attention Mask:", inputs['attention_mask'])

print( f'labels: {labels}' )

print(f'inputs: {dict(inputs)}')

Input IDs: tf.Tensor(
[[  101 12548 10127 11153 16869 13347   136   102     0     0     0     0]
 [  101 11523 10127 11153 31117   136   102     0     0     0     0     0]
 [  101 16118 10525 84408 10114 14743 10111 10525 21343 10700 15033   102]
 [  101 12548 10743   151 11481   143 10246 21157   136   102     0     0]
 [  101 10743   151 15130 11153 89933   136   102     0     0     0     0]
 [  101 12548 10154   151 24531 11153 19089 11285   136   102     0     0]
 [  101   151 16612 10114 13780 11153 21157 12340   119   102     0     0]], shape=(7, 12), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 1 1 1 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0]], shape=(7, 12), dtype=int32)
labels: [0 1 2 3 4 5 6]
inputs: {'input_ids': <tf.Tensor: shape=(7, 12), dtype=int32, numpy=
array([[  101, 12548, 10127, 11153, 16869, 13347,   136,   102,     0,
            0

### Fine-tuning the Model

In [35]:
model = TFBertForSequenceClassification.from_pretrained( 'bert-base-multilingual-uncased', num_labels=7 )

model.compile( optimizer=Adam( learning_rate=5e-5 ), loss=model.compute_loss, metrics=['accuracy'] )

model.fit( dict(inputs), labels, batch_size=8, epochs=10 )

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


AttributeError: in user code:

    File "/home/oszymanski/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/oszymanski/.local/lib/python3.10/site-packages/transformers/modeling_tf_utils.py", line 1588, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "/home/oszymanski/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss  **
        return self.compiled_loss(
    File "/home/oszymanski/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 275, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/home/oszymanski/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 854, in match_dtype_and_rank
        if (y_t.dtype.is_floating and y_p.dtype.is_floating) or (

    AttributeError: 'NoneType' object has no attribute 'dtype'


### Handle intentions and backend logic

In [18]:
def predict_intent( user_input ):
    inputs = tokenizer( user_input, return_tensors='tf', padding=True, truncation=True )

    outputs = model( inputs )
    predicted_label_idx = tf.argmax( outputs.logits, axis=1 ).numpy()[0]
    predicted_label = id_to_label[predicted_label_idx]

    return predicted_label


predicted_label = predict_intent( 'Can I close my savings?' )
print( f'predicted intent: { predicted_label }' )

predicted intent: change_account_type
